In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Liver_cirrhosis"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Liver_cirrhosis/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Liver_cirrhosis/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Liver_cirrhosis/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Liver_cirrhosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os

# Check if there's a suitable cohort directory for Liver_cirrhosis
print(f"Looking for a relevant cohort directory for {trait}...")

# Check available cohorts
available_dirs = os.listdir(tcga_root_dir)
print(f"Available cohorts: {available_dirs}")

# Liver cirrhosis is a liver condition, so we should look for liver-related cohorts
liver_related_dirs = [d for d in available_dirs if any(term in d.lower() for term in ['liver', 'hepat', 'lihc'])]
print(f"Liver-related cohorts: {liver_related_dirs}")

# Select the most relevant cohort for liver cirrhosis
if liver_related_dirs:
    selected_cohort = liver_related_dirs[0]  # TCGA_Liver_Cancer_(LIHC) is the most relevant
    print(f"Selected cohort: {selected_cohort}")
    
    # Get the full path to the selected cohort directory
    cohort_dir = os.path.join(tcga_root_dir, selected_cohort)
    
    # Get the clinical and genetic data file paths
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical data file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic data file: {os.path.basename(genetic_file_path)}")
    
    # Load the clinical and genetic data
    clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
    genetic_df = pd.read_csv(genetic_file_path, index_col=0, sep='\t')
    
    # Print the column names of the clinical data
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Basic info about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")
    
else:
    print(f"No liver-related cohorts found for {trait}.")
    # Mark the task as completed by recording the unavailability in the cohort_info.json file
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )


Looking for a relevant cohort directory for Liver_cirrhosis...
Available cohorts: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 


Clinical data columns:
['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'adjacent_hepatic_tissue_inflammation_extent_type', 'age_at_initial_pathologic_diagnosis', 'albumin_result_lower_limit', 'albumin_result_specified_value', 'albumin_result_upper_limit', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'bilirubin_lower_limit', 'bilirubin_upper_limit', 'cancer_first_degree_relative', 'child_pugh_classification_grade', 'creatinine_lower_level', 'creatinine_upper_limit', 'creatinine_value_in_mg_dl', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'eastern_cancer_oncology_group', 'fetoprotein_outcome_lower_limit', 'fetoprotein_outcome_upper_limit', 'fetoprotein_outcome_value', 'fibrosis_ishak_sc

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify candidate columns for age and gender
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
candidate_gender_cols = ['gender']

# Find the clinical data file path and load the data
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Liver_Cancer_(LIHC)')
clinical_file_path, _ = tcga_get_relevant_filepaths(cohort_dir)
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Extract and preview age data
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].head(5).tolist()

# Extract and preview gender data
gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].head(5).tolist()

print("Candidate age columns:", candidate_age_cols)
print("Age preview:", age_preview)
print("Candidate gender columns:", candidate_gender_cols)
print("Gender preview:", gender_preview)


Candidate age columns: ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
Age preview: {'age_at_initial_pathologic_diagnosis': [nan, 58.0, 51.0, 55.0, 54.0], 'days_to_birth': [nan, -21318.0, -18768.0, -20187.0, -20011.0]}
Candidate gender columns: ['gender']
Gender preview: {'gender': ['MALE', 'MALE', 'MALE', 'FEMALE', 'FEMALE']}


### Step 3: Select Demographic Features

In [4]:
# Examine candidate columns for age and gender

# For age information, we have two candidates:
# 1. age_at_initial_pathologic_diagnosis - contains the age in years directly
# 2. days_to_birth - contains negative values representing days from birth to diagnosis

# For gender information, we only have one candidate:
# 1. gender - contains values like 'MALE' and 'FEMALE'

# Let's select the appropriate columns:

# For age, 'age_at_initial_pathologic_diagnosis' is more straightforward as it directly gives age in years
# The values look reasonable (between 51-58 years old for the first 5 samples)
age_col = 'age_at_initial_pathologic_diagnosis'

# For gender, we only have one option and the values look appropriate
gender_col = 'gender'

# Print out the chosen columns
print(f"Chosen age column: {age_col}")
print(f"Chosen gender column: {gender_col}")


Chosen age column: age_at_initial_pathologic_diagnosis
Chosen gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# 1. Extract and standardize clinical features
# Use tcga_select_clinical_features which will automatically create the trait variable and add age/gender if provided
linked_clinical_df = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# Print preview of clinical features
print("Clinical features (first 5 rows):")
print(linked_clinical_df.head())

# 2. Process gene expression data
print("\nProcessing gene expression data...")
# Load genetic data if not already loaded
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Liver_Cancer_(LIHC)')
_, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Check gene data shape
print(f"Original gene data shape: {genetic_df.shape}")

# Save a version of the gene data before normalization (as a backup)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
genetic_df.to_csv(out_gene_data_file.replace('.csv', '_original.csv'))

# We need to transpose genetic data so genes are rows and samples are columns for normalization
gene_df_for_norm = genetic_df.copy().T

# Try to normalize gene symbols - adding debug output to understand what's happening
print("Attempting to normalize gene symbols...")
try:
    normalized_gene_df = normalize_gene_symbols_in_index(gene_df_for_norm)
    print(f"Gene data shape after normalization: {normalized_gene_df.shape}")
    
    # Check if normalization returned empty DataFrame
    if normalized_gene_df.shape[0] == 0:
        print("WARNING: Gene symbol normalization returned an empty DataFrame.")
        print("Using original gene data instead of normalized data.")
        # Use original data instead - samples as rows, genes as columns
        normalized_gene_df = genetic_df
    else:
        # If normalization worked, transpose back to original orientation
        normalized_gene_df = normalized_gene_df.T
except Exception as e:
    print(f"Error during gene symbol normalization: {e}")
    print("Using original gene data instead.")
    normalized_gene_df = genetic_df

# Save gene data
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Gene data saved to: {out_gene_data_file}")

# 3. Link clinical and genetic data
# TCGA data uses the same sample IDs in both datasets
print("\nLinking clinical and genetic data...")
print(f"Clinical data shape: {linked_clinical_df.shape}")
print(f"Genetic data shape: {normalized_gene_df.shape}")

# Find common samples between clinical and genetic data
common_samples = set(linked_clinical_df.index).intersection(set(normalized_gene_df.columns))
print(f"Number of common samples: {len(common_samples)}")

if len(common_samples) == 0:
    print("ERROR: No common samples found between clinical and genetic data.")
    # Use is_final=False mode which doesn't require df and is_biased
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True
    )
    print("The dataset was determined to be unusable for this trait due to no common samples. No data files were saved.")
else:
    # Filter clinical data to only include common samples
    linked_clinical_df = linked_clinical_df.loc[list(common_samples)]
    
    # Create linked data by merging
    linked_data = pd.concat([linked_clinical_df, normalized_gene_df[list(common_samples)].T], axis=1)
    
    print(f"\nLinked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, first few columns):")
    display_cols = [trait, 'Age', 'Gender'] + list(linked_data.columns[3:5])
    print(linked_data[display_cols].head())
    
    # 4. Handle missing values
    linked_data = handle_missing_values(linked_data, trait)
    print(f"\nData shape after handling missing values: {linked_data.shape}")
    
    # 5. Check for bias in trait and demographic features
    print("\nChecking for bias in features:")
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 6. Validate and save cohort info
    print("\nPerforming final validation...")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=len(linked_data.columns) > 3,  # More than just trait/age/gender columns
        is_trait_available=trait in linked_data.columns,
        is_biased=is_trait_biased,
        df=linked_data,
        note="Data from TCGA Liver Cancer (LIHC) cohort used as proxy for Liver Cirrhosis."
    )
    
    # 7. Save linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to: {out_data_file}")
        
        # Also save clinical data separately
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_columns = [col for col in linked_data.columns if col in [trait, 'Age', 'Gender']]
        linked_data[clinical_columns].to_csv(out_clinical_data_file)
        print(f"Clinical data saved to: {out_clinical_data_file}")
    else:
        print("The dataset was determined to be unusable for this trait. No data files were saved.")

Clinical features (first 5 rows):
                 Liver_cirrhosis   Age  Gender
sampleID                                      
TCGA-2V-A95S-01                1   NaN       1
TCGA-2Y-A9GS-01                1  58.0       1
TCGA-2Y-A9GT-01                1  51.0       1
TCGA-2Y-A9GU-01                1  55.0       0
TCGA-2Y-A9GV-01                1  54.0       0

Processing gene expression data...


Original gene data shape: (20530, 423)


Attempting to normalize gene symbols...
Gene data shape after normalization: (0, 20530)
Using original gene data instead of normalized data.


Gene data saved to: ../../output/preprocess/Liver_cirrhosis/gene_data/TCGA.csv

Linking clinical and genetic data...
Clinical data shape: (438, 3)
Genetic data shape: (20530, 423)
Number of common samples: 423

Linked data shape: (423, 20533)
Linked data preview (first 5 rows, first few columns):
                 Liver_cirrhosis   Age  Gender  ARHGEF10L     HIF3A
TCGA-RC-A7SF-01                1  66.0       1   2.798308 -2.589526
TCGA-DD-AAVR-01                1  44.0       1   1.278708 -1.048626
TCGA-G3-A5SK-01                1  58.0       1   1.556508 -2.907826
TCGA-RC-A7SH-01                1  42.0       1   1.871108 -4.012626
TCGA-CC-5262-01                1  67.0       1   1.700608 -2.261426



Data shape after handling missing values: (423, 20533)

Checking for bias in features:
For the feature 'Liver_cirrhosis', the least common label is '0' with 50 occurrences. This represents 11.82% of the dataset.
The distribution of the feature 'Liver_cirrhosis' in this dataset is fine.

Quartiles for 'Age':
  25%: 52.0
  50% (Median): 62.0
  75%: 69.0
Min: 16.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 143 occurrences. This represents 33.81% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.


Performing final validation...


Linked data saved to: ../../output/preprocess/Liver_cirrhosis/TCGA.csv
Clinical data saved to: ../../output/preprocess/Liver_cirrhosis/clinical_data/TCGA.csv
